# Mobile Metrics

Loads and saves webpagetest.org performance tests.

Needs a local installation of the webpagetest tools https://github.com/WPO-Foundation/webpagetest and a mobile device or a key for the webpagetest API.

A complete test may last for up to 2 hours.

Feel free to contact me for help: https://www.quel-media.com/about.html#contact

© Paul Ronga under Apache-2 Licence (see LICENCE.txt).

In [1]:
import pandas as pd
import requests
from IPython.display import HTML
import json
import datetime

In [2]:
# Link to local PHP files
RUN_TEST_URL = 'http://rospo.local/~paul/webpagetest/run_mobile.php'
GET_TEST_URL = 'http://rospo.local/~paul/webpagetest/get.php?test_id={}'

In [3]:
# dataframe containing media id, name and URLs
medias = pd.read_csv('df/media_list.csv')

# remove Konbini ch and fr
medias = medias[medias['media_id'] < 34].copy()

# media id as string
medias['media_id'] = medias['media_id'].apply(lambda x: str(x))

medias.head(2)

,media_id,Name,URL_short,URL,URL_mobile
0,19,La Tribune de Genève,tdg.ch,https://www.tdg.ch/,https://m.tdg.ch
1,20,24 heures,24heures.ch,https://www.24heures.ch,https://m.24heures.ch


# 1. Lauch tests

Lauch tests (average of 3 runs) for each media on a device monitored with the webpagetest system.

In [4]:
reports = {}

In [5]:
for i, row in medias.iterrows():
    url = row['URL_mobile']
    if url != url or url == 'x':
        url = row['URL']

    print('Running test for', url)

    payload = {'url': url}
    r = requests.post(RUN_TEST_URL, json=payload)
    
    print(r.text, end='\n\n')
    result = r.json()
    reports[row['media_id']] = result['id']

Running test for https://m.tdg.ch
{"id":"180901_SN_d60f244f03eb854b7c22053f3d988b1e", "url":"https://m.tdg.ch"}

Running test for https://m.24heures.ch
{"id":"180901_KP_ae80a978d10dea8fa9951f2e70f5e298", "url":"https://m.24heures.ch"}

Running test for https://www.letemps.ch
{"id":"180901_N7_edf1e828cfe8e624a3f1f20138b84303", "url":"https://www.letemps.ch"}

Running test for https://mobile.lemonde.fr
{"id":"180901_GQ_a06ba67036d91df7918469ca8da325b5", "url":"https://mobile.lemonde.fr"}

Running test for https://www.rts.ch/info/
{"id":"180901_7Y_c09e564ea1ad82acd95af1cb7f9c3510", "url":"https://www.rts.ch/info/"}

Running test for https://m.20min.ch/ro/
{"id":"180901_YD_3464e9558090165e0feada65218276ba", "url":"https://m.20min.ch/ro/"}

Running test for https://m.lematin.ch
{"id":"180901_KT_41eac205e45284841cde1652f69cc66a", "url":"https://m.lematin.ch"}

Running test for https://www.mediapart.fr
{"id":"180901_HM_bb052bfb3d116300e2773c3cf5fe4622", "url":"https://www.mediapart.fr"}

Runn

In [6]:
reports

{'19': '180901_SN_d60f244f03eb854b7c22053f3d988b1e',
 '20': '180901_KP_ae80a978d10dea8fa9951f2e70f5e298',
 '21': '180901_N7_edf1e828cfe8e624a3f1f20138b84303',
 '22': '180901_GQ_a06ba67036d91df7918469ca8da325b5',
 '23': '180901_7Y_c09e564ea1ad82acd95af1cb7f9c3510',
 '24': '180901_YD_3464e9558090165e0feada65218276ba',
 '25': '180901_KT_41eac205e45284841cde1652f69cc66a',
 '26': '180901_HM_bb052bfb3d116300e2773c3cf5fe4622',
 '27': '180901_PA_167b2086f17e1504aabe3f7b2605739a',
 '28': '180901_VE_5d709bcdfd718605f74b79aab50052b2',
 '29': '180901_N4_b037d014dc2ebf5674f4d5762b122271',
 '30': '180901_0S_633feec82b96c76be886078c8245492a',
 '31': '180901_2W_5f0283b241d63dbf519f600b12ba9b98',
 '32': '180901_X3_c72b595230502c699873bb7af16cef32',
 '33': '180901_ME_5d9f584769e1619e0ebc7bdcea483634'}

In [7]:
# backup report ids
with open('reports.json', 'w') as outfile:
    json.dump(reports, outfile)

# 2. Get test results

Wait about one hour before trying to get these results.

In [8]:
df_speed = pd.DataFrame(columns=['Name', 'media_id', 'location', 'visualComplete', 'fullyLoaded', 'requests', 'report_url', 'completed'])

In [9]:
from IPython.display import HTML

In [11]:
for i, row in medias.iterrows():
    print('Getting results for', row['Name'], '...')
    
    if len(df_speed[df_speed['media_id'] == row['media_id']]) > 0:
        print('Result already fetched.')
    else:    
        r = requests.get(GET_TEST_URL.format( reports[row['media_id']] ))
        print(r)
        try:
            data = r.json()
        except ValueError as e:
            print(e)
            print(r.text)

        missing_key = False

        if 'data' in data.keys() and 'average' in data['data'].keys():    
            print('Successful first view runs:', data['data']['successfulFVRuns'])

            print('Location and device were:', data['data']['location'], 'with connectivity:', data['data']['connectivity'])
            
            for key in ['visualComplete', 'fullyLoaded', 'requests']:
                if key not in data['data']['average']['firstView'].keys():
                    print('Missing key:', key)
                    print(data['data']['average']['firstView'].keys())
                    if key == 'visualComplete' and 'fullyLoaded' in data['data']['average']['firstView'].keys():
                        data['data']['average']['firstView']['visualComplete'] = data['data']['average']['firstView']['fullyLoaded']
                    else:
                        missing_key = True

            if not missing_key:
                df_speed = df_speed.append(pd.DataFrame([[
                    row['Name'],
                    row['media_id'],
                    data['data']['location'],
                    int(data['data']['average']['firstView']['visualComplete']),
                    int(data['data']['average']['firstView']['fullyLoaded']),
                    int(data['data']['average']['firstView']['requests']),
                    data['data']['summary'],
                    data['data']['completed']
                ]], columns=['Name', 'media_id', 'location', 'visualComplete', 'fullyLoaded', 'requests', 'report_url', 'completed']))
            
                with open('df/archive/mobile/{}.json'.format(data['data']['testId']), 'w') as outfile:
                    json.dump(data['data'], outfile)
                print('Saved to disk and json stored in df/archive/mobile/', end='\n\n')
        else:
            print('No result for {} yet.'.format(row['Name']))

Getting results for La Tribune de Genève ...
Result already fetched.
Getting results for 24 heures ...
Result already fetched.
Getting results for Le Temps ...
Result already fetched.
Getting results for Le Monde ...
Result already fetched.
Getting results for RTS info ...
Result already fetched.
Getting results for 20 minutes (ch) ...
Result already fetched.
Getting results for Le Matin ...
Result already fetched.
Getting results for Mediapart ...
Result already fetched.
Getting results for Le Figaro ...
Result already fetched.
Getting results for Libération ...
Result already fetched.
Getting results for La Côte ...
Result already fetched.
Getting results for Arcinfo ...
Result already fetched.
Getting results for Le Nouvelliste ...
Result already fetched.
Getting results for La Liberté ...
Result already fetched.
Getting results for Le Courrier ...
Result already fetched.


In [13]:
df_speed.sort_values('fullyLoaded')
#df_speed.sort_values('visualComplete')

,Name,media_id,location,visualComplete,fullyLoaded,requests,report_url,completed
0,Mediapart,26,Dulles_iPhone8:iPhone 8 iOS 11,12088,15853,100,http://www.webpagetest.org/result/180901_HM_bb...,"Sat, 01 Sep 2018 16:40:14 +0000"
0,Le Figaro,27,Dulles_iPhone8:iPhone 8 iOS 11,12088,15853,100,http://www.webpagetest.org/result/180901_HM_bb...,"Sat, 01 Sep 2018 16:40:14 +0000"
0,Libération,28,Dulles_iPhone8:iPhone 8 iOS 11,12088,15853,100,http://www.webpagetest.org/result/180901_HM_bb...,"Sat, 01 Sep 2018 16:40:14 +0000"
0,La Côte,29,Dulles_iPhone8:iPhone 8 iOS 11,12088,15853,100,http://www.webpagetest.org/result/180901_HM_bb...,"Sat, 01 Sep 2018 16:40:14 +0000"
0,Le Courrier,33,Dulles_iPhone8:iPhone 8 iOS 11,16414,16792,70,http://www.webpagetest.org/result/180901_ME_5d...,"Sat, 01 Sep 2018 17:26:54 +0000"
0,Le Matin,25,Dulles_iPhone8:iPhone 8 iOS 11,14242,16866,227,http://www.webpagetest.org/result/180901_KT_41...,"Sat, 01 Sep 2018 16:36:29 +0000"
0,24 heures,20,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000"
0,Le Temps,21,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000"
0,Le Monde,22,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000"
0,RTS info,23,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000"


In [14]:
df_speed['mobile_index'] = df_speed['fullyLoaded'].apply(lambda x: round(100 - (int(x)-5000)/1000))
df_speed.sort_values('mobile_index')

,Name,media_id,location,visualComplete,fullyLoaded,requests,report_url,completed,mobile_index
0,La Liberté,32,Dulles_iPhone8:iPhone 8 iOS 11,27094,31657,154,http://www.webpagetest.org/result/180901_X3_c7...,"Sat, 01 Sep 2018 17:15:45 +0000",73
0,La Tribune de Genève,19,Dulles_iPhone8:iPhone 8 iOS 11,17718,21070,215,http://www.webpagetest.org/result/180901_SN_d6...,"Sat, 01 Sep 2018 16:06:14 +0000",84
0,Arcinfo,30,Dulles_iPhone8:iPhone 8 iOS 11,9351,20966,138,http://www.webpagetest.org/result/180901_0S_63...,"Sat, 01 Sep 2018 17:02:41 +0000",84
0,24 heures,20,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000",86
0,Le Temps,21,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000",86
0,Le Monde,22,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000",86
0,RTS info,23,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000",86
0,20 minutes (ch),24,Dulles_iPhone8:iPhone 8 iOS 11,16554,18571,274,http://www.webpagetest.org/result/180901_KP_ae...,"Sat, 01 Sep 2018 16:12:27 +0000",86
0,Le Nouvelliste,31,Dulles_iPhone8:iPhone 8 iOS 11,15705,18601,153,http://www.webpagetest.org/result/180901_2W_5f...,"Sat, 01 Sep 2018 17:07:51 +0000",86
0,Le Matin,25,Dulles_iPhone8:iPhone 8 iOS 11,14242,16866,227,http://www.webpagetest.org/result/180901_KT_41...,"Sat, 01 Sep 2018 16:36:29 +0000",88


In [15]:
outputfile = 'df/archive/mobile_metrics_{}.csv'.format( datetime.datetime.now().strftime('%Y-%m-%d') )
print('Saving to {}...'.format(outputfile))

Saving to df/archive/mobile_metrics_2018-09-02.csv...


In [16]:
df_speed.to_csv(outputfile) # archive
df_speed.to_csv('df/mobile_metrics.csv') # temp file

In [17]:
medias

,media_id,Name,URL_short,URL,URL_mobile
0,19,La Tribune de Genève,tdg.ch,https://www.tdg.ch/,https://m.tdg.ch
1,20,24 heures,24heures.ch,https://www.24heures.ch,https://m.24heures.ch
2,21,Le Temps,letemps.ch,https://www.letemps.ch,NaN
3,22,Le Monde,lemonde.fr,https://www.lemonde.fr,https://mobile.lemonde.fr
4,23,RTS info,rts.ch/info,https://www.rts.ch/info/,NaN
5,24,20 minutes (ch),20min.ch/ro,https://www.20min.ch/ro/,https://m.20min.ch/ro/
6,25,Le Matin,lematin.ch,https://www.lematin.ch/,https://m.lematin.ch
7,26,Mediapart,mediapart.fr,https://www.mediapart.fr,x
8,27,Le Figaro,lefigaro.fr,https://www.lefigaro.fr/,x
9,28,Libération,liberation.fr,https://www.liberation.fr/,NaN


### stop here

In [15]:
#df = pd.read_csv('/Users/paul/Sites/d3_v5/indices/data/media_metrics_2018-06-17.csv')

In [16]:
#df_mobile = pd.read_csv('df/mobile_metrics.csv', usecols=['media_id', 'location', 'visualComplete', 'fullyLoaded',
       'requests', 'report_url', 'mobile_index'])

In [17]:
#df.merge(df_mobile, on='media_id').to_csv('/Users/paul/Sites/d3_v5/indices/data/media_metrics_2018-06-17.csv')

### NB

** This script only uses loading time. Optimization is evaluated with PageSpeed. But these 9 scores are also available: **
```'score_cache',
'score_cdn',
'score_combine',
'score_compress',
'score_cookies',
'score_etags',
'score_gzip',
'score_keep-alive',
'score_minify'```